In [1]:
from llm_providers import GoogleAIModel
import os
from dotenv import load_dotenv
from multi_agent import SyntheticDataGenerator
from langchain_community.retrievers import BM25Retriever
from utils import pdf_parser
from tasks import Task

load_dotenv()
key = os.environ.get('GOOGLE_API_KEY')
llm = GoogleAIModel(api_key=key)
pdf_files = pdf_parser('cv.pdf')

bm25retriever = BM25Retriever.from_documents(pdf_files, k=5)

agent = SyntheticDataGenerator(llm=llm, 
                               retriever=bm25retriever,
                               output_path='./output',
                               buffer_size=5)

example_task = Task(
    task_name="dpo",
    localization='Optimization, Data Science and AI',
    task_description="Generate a conversation between a HR manager and a candidate applying for a data science position using this information.",
    rows_per_batch=11,
    batch_size=3,
    language="Vietnamese"
)

╭───────────────────────────────────────────────────── INFO ──────────────────────────────────────────────────────╮
│ Calling pdf_parser                                                                                              │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── INFO ──────────────────────────────────────────────────────╮
│ Extracted 2 pages. First page preview:                                                                          │
│ ##############                                                                                                  │
│ PHAM KINH QUOC                                                                                                  │
│ +81 80-4721-4151• phamkinhquoc2002@gmail.com • https://www.linkedin.com/in/kinh-quốc-...                        │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── INFO ──────────────────────────────────────────────────────╮
│ Loading 2 pieces of context!                                                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── INFO ──────────────────────────────────────────────────────╮
│ Split successful. Last chunk preview:                                                                           │
│ ##############                                                                                                  │
│ SPECIALIZATION                                                                                                  │
│ • LLM: Retrieval Augmented Generation, Multi-Agentic Collaboration, MCP, Reasoning LL...                        │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [2]:
thread_config = {'configurable': {'thread_id': 123}}

async for output in agent.agent_flow.astream(
    {
        "task": example_task,
        "human_feedback": None,
        "conversations": []
    }
    ,
    config=thread_config, 
    stream_mode="updates"):
    for key, value in output.items():
        print(value)
    print("\n---\n")

╭──────────────────────────────────────────────── OUTPUT_MESSAGE ─────────────────────────────────────────────────╮
│ FORMATTED DOCUMENTS PREVIEW:                                                                                    │
│ ##############                                                                                                  │
│                                                                                                                 │
│ - SPECIALIZATION                                                                                                │
│ • LLM: Retrieval Augmented Generation, Multi-Agentic Collaboration, MCP, Reasoning LLMs,Instruction             │
│ Finetuning (Lora, QLora, RLHF, DPO, DDP, FSDP), Knowledge Graphs, Multimodal, Dist                              │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

{'retrieved_documents': ['\n- SPECIALIZATION\n• LLM: Retrieval Augmented Generation, Multi-Agentic Collaboration, MCP, Reasoning LLMs,Instruction\nFinetuning (Lora, QLora, RLHF, DPO, DDP, FSDP), Knowledge Graphs, Multimodal, Distributed Training\n(Model Parallelism, Data Parallelism).\n• Data Engineering: ETL Pipelines, CI/CD Pipelines, DevOps, Data Warehouse, Distributed Data Processing,\nWorkflow Orchestration.\n• Cloud and Infrastructure: AWS, Google Cloud, Docker, Kubernetes, CUDA.\n• Programming Language and other tools: Python, C++, Javascript, Pytorch, Git, Linux, Terraform, Neo4j,\nPostgre, MySQL, Kestra, Apache Spark, Apache Kafka, Redis, Prometheus, Grafana.\n• Notable Frameworks: TRL, langchain, llama-index, autogen, crewAI, smolAgents, fastapi, TensorRTX, vllm.\n• Domain Knowledge: MLOPs, LLM system, Data Engineering.\nCERTIFICATIONS AND COMPLETED COURSES\nIntroduction to Machine Learning by University of Munich Completed Course- 2023\nThis course provides foundational know

╭───────────────────────────────────────────────────── INFO ──────────────────────────────────────────────────────╮
│ Total number of conversation turns up to this point: 0                                                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── INFO ──────────────────────────────────────────────────────╮
│ Calling prompt_initialize                                                                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── OUTPUT_MESSAGE ─────────────────────────────────────────────────╮
│                                                                                                                 │
│ ---------FISH_FOR_FEEDBACK---------                                                                             │
│                                                                                                                 │
│ System Prompt:                                                                                                  │
│ You are an advanced synthetic data generator, engineered to produce high-quality, task-specific synthetic       │
│ datasets. Your mission is to generate data samples in formats that precisely adhere to the requirements         │
│ provided.                                                                                                       │
│                                                                                                                 │
│ User Prompt:You are tasked to help me generate a dataset of 2 rows entirely in Vietnamese, based entirely on    │
│ the following context:                                                                                          │
│ - SPECIALIZATION                                                                                                │
│ • LLM: Retrieval Augmented Generation, Multi-Agentic Collaboration, MCP, Reasoning LLMs,Instruction             │
│ Finetuning (Lora, QLora, RLHF, DPO, DDP, FSDP), Knowledge Graphs, Multimodal, Distributed Training              │
│ (Model Parallelism, Data Parallelism).                                                                          │
│ • Data Engineering: ETL Pipelines, CI/CD Pipelines, DevOps, Data Warehouse, Distributed Data Processing,        │
│ Workflow Orchestration.                                                                                         │
│ • Cloud and Infrastructure: AWS, Google Cloud, Docker, Kubernetes, CUDA.                                        │
│ • Programming Language and other tools: Python, C++, Javascript, Pytorch, Git, Linux, Terraform, Neo4j,         │
│ Postgre, MySQL, Kestra, Apache Spark, Apache Kafka, Redis, Prometheus, Grafana.                                 │
│ • Notable Frameworks: TRL, langchain, llama-index, autogen, crewAI, smolAgents, fastapi, TensorRTX, vllm.       │
│ • Domain Knowledge: MLOPs, LLM system, Data Engineering.                                                        │
│ CERTIFICATIONS AND COMPLETED COURSES                                                                            │
│ Introduction to Machine Learning by University of Munich Completed Course- 2023                                 │
│ This course provides foundational knowledge to machine learning.                                                │
│ Deep Learning Specialization by Andrew Ng Completed Course- 2023                                                │
│ Curated by renowned AI expert, Andrew Ng. It helped me delve into the foundations of deep learning.             │
│ Generative AI with Large Language Models Certification- 2024                                                    │
│ A course provided by DeepLearning.AI. I learned about Generative AI project lifecycle, Prompt Engineering,      │
│ fine-tuning and evaluation, Reinforcement Learning and LLM-powered applications.                                │
│ MLOPS with AWS - Bootcamp Certification- 2024                                                                   │
│ A Practical course that helps me build Machine Learning pipeline in AWS.                                        │
│ NEO4J-powered LLM chatbot with Python Certification- 2024                                                       │
│ A course provided by Neo4j to help me build a LLM-powered backend system using knowledge graphs.                │
│ AWARDS                                                

╭──────────────────────────────────────────────── OUTPUT_MESSAGE ─────────────────────────────────────────────────╮
│                                                                                                                 │
│ ---------FISH_FOR_FEEDBACK---------                                                                             │
│                                                                                                                 │
│ Response:```json                                                                                                │
│ [                                                                                                               │
│   {                                                                                                             │
│     "prompt": "Hãy đóng vai một nhà quản lý nhân sự phỏng vấn một ứng viên cho vị trí khoa học dữ liệu. Hỏi về  │
│ kinh nghiệm của ứng viên trong các lĩnh vực như LLM, Data Engineering, Cloud và Infrastructure, cũng như các    │
│ công cụ và framework mà họ quen thuộc. Đề cập đến các chứng chỉ và khóa học mà ứng viên đã hoàn thành, đặc biệt │
│ là các khóa học liên quan đến Machine Learning, Deep Learning, Generative AI và MLOps. Cuối cùng, hỏi về các    │
│ giải thưởng và thành tích mà ứng viên đã đạt được.",                                                            │
│     "chosen": "Chào mừng bạn đến với buổi phỏng vấn cho vị trí khoa học dữ liệu tại công ty chúng tôi. Rất vui  │
│ được gặp bạn. Để bắt đầu, bạn có thể chia sẻ về kinh nghiệm của mình trong các lĩnh vực như LLM, Data           │
│ Engineering, Cloud và Infrastructure không? Chúng tôi đặc biệt quan tâm đến kinh nghiệm của bạn với Retrieval   │
│ Augmented Generation, Multi-Agentic Collaboration, MCP, Reasoning LLMs, Instruction Finetuning, Knowledge       │
│ Graphs, Multimodal và Distributed Training. Bạn có kinh nghiệm với ETL Pipelines, CI/CD Pipelines, DevOps, Data │
│ Warehouse, Distributed Data Processing và Workflow Orchestration không? Về Cloud và Infrastructure, bạn có kinh │
│ nghiệm với AWS, Google Cloud, Docker, Kubernetes và CUDA không? Bạn có thể cho biết thêm về các công cụ và      │
│ framework mà bạn quen thuộc, chẳng hạn như Python, C++, Javascript, Pytorch, Git, Linux, Terraform, Neo4j,      │
│ Postgre, MySQL, Kestra, Apache Spark, Apache Kafka, Redis, Prometheus, Grafana, TRL, langchain, llama-index,    │
│ autogen, crewAI, smolAgents, fastapi, TensorRTX và vllm không? Tôi thấy bạn đã hoàn thành một số chứng chỉ và   │
│ khóa học, bao gồm Introduction to Machine Learning, Deep Learning Specialization, Generative AI with Large      │
│ Language Models Certification và MLOPS with AWS - Bootcamp Certification. Bạn có thể chia sẻ thêm về những gì   │
│ bạn đã học được từ những khóa học này và cách chúng áp dụng vào công việc của chúng tôi không? Cuối cùng, tôi   │
│ thấy bạn đã đạt được một số giải thưởng và thành tích, bao gồm Hello GPT-4o Hackathon Certificate of            │
│ Completion, Assistants API and Trulens Hackathon Certificate of Completion, Second Prize - Mathematics in       │
│ English at district level và Consolation Prize - Hanoi Open Math Competition. Bạn có thể chia sẻ thêm về những  │
│ thành tích này và cách chúng thể hiện khả năng của bạn không?",                                                 │
│     "rejected": "Chào bạn, hãy nói về kinh nghiệm của bạn."                                                     │
│   },                                                                                                            │
│   {                                                                                                             │
│     "prompt": "Hãy đóng vai một ứng viên phỏng vấn cho vị trí khoa học dữ liệu. Trả lời các câu hỏi của nhà     │
│ quản lý nhân sự về kinh nghiệm của bạn trong các lĩnh vực như LLM, Data Engineering, Cloud và Infrastructure,   │
│ cũng như các công cụ và framework mà bạn quen thuộc. N

{'response': '```json\n[\n  {\n    "prompt": "Hãy đóng vai một nhà quản lý nhân sự phỏng vấn một ứng viên cho vị trí khoa học dữ liệu. Hỏi về kinh nghiệm của ứng viên trong các lĩnh vực như LLM, Data Engineering, Cloud và Infrastructure, cũng như các công cụ và framework mà họ quen thuộc. Đề cập đến các chứng chỉ và khóa học mà ứng viên đã hoàn thành, đặc biệt là các khóa học liên quan đến Machine Learning, Deep Learning, Generative AI và MLOps. Cuối cùng, hỏi về các giải thưởng và thành tích mà ứng viên đã đạt được.",\n    "chosen": "Chào mừng bạn đến với buổi phỏng vấn cho vị trí khoa học dữ liệu tại công ty chúng tôi. Rất vui được gặp bạn. Để bắt đầu, bạn có thể chia sẻ về kinh nghiệm của mình trong các lĩnh vực như LLM, Data Engineering, Cloud và Infrastructure không? Chúng tôi đặc biệt quan tâm đến kinh nghiệm của bạn với Retrieval Augmented Generation, Multi-Agentic Collaboration, MCP, Reasoning LLMs, Instruction Finetuning, Knowledge Graphs, Multimodal và Distributed Training. B

In [3]:
from langgraph.types import Command

await agent.agent_flow.ainvoke(
    Command(resume='yes'),
    config=thread_config
)

╭───────────────────────────────────────────────────── INFO ──────────────────────────────────────────────────────╮
│ Calling prompt_initialize                                                                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── INFO ──────────────────────────────────────────────────────╮
│ Calling one_shot_prompt                                                                                         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── OUTPUT_MESSAGE ─────────────────────────────────────────────────╮
│ You are tasked to help me generate a dataset of 11 rows entirely in Vietnamese, based entirely on the following │
│ context:                                                                                                        │
│ - SPECIALIZATION                                                                                                │
│ • LLM: Retrieval Augmented Generation, Multi-Agentic Collaboration, MCP, Reasoning LLMs,Instruction             │
│ Finetuning (Lora, QLora, RLHF, DPO, DDP, FSDP), Knowledge Graphs, Multimodal, Distributed Training              │
│ (Model Parallelism, Data Parallelism).                                                                          │
│ • Data Engineering: ETL Pipelines, CI/CD Pipelines, DevOps, Data Warehouse, Distributed Data Processing,        │
│ Workflow Orchestration.                                                                                         │
│ • Cloud and Infrastructure: AWS, Google Cloud, Docker, Kubernetes, CUDA.                                        │
│ • Programming Language and other tools: Python, C++, Javascript, Pytorch, Git, Linux, Terraform, Neo4j,         │
│ Postgre, MySQL, Kestra, Apache Spark, Apache Kafka, Redis, Prometheus, Grafana.                                 │
│ • Notable Frameworks: TRL, langchain, llama-index, autogen, crewAI, smolAgents, fastapi, TensorRTX, vllm.       │
│ • Domain Knowledge: MLOPs, LLM system, Data Engineering.                                                        │
│ CERTIFICATIONS AND COMPLETED COURSES                                                                            │
│ Introduction to Machine Learning by University of Munich Completed Course- 2023                                 │
│ This course provides foundational knowledge to machine learning.                                                │
│ Deep Learning Specialization by Andrew Ng Completed Course- 2023                                                │
│ Curated by renowned AI expert, Andrew Ng. It helped me delve into the foundations of deep learning.             │
│ Generative AI with Large Language Models Certification- 2024                                                    │
│ A course provided by DeepLearning.AI. I learned about Generative AI project lifecycle, Prompt Engineering,      │
│ fine-tuning and evaluation, Reinforcement Learning and LLM-powered applications.                                │
│ MLOPS with AWS - Bootcamp Certification- 2024                                                                   │
│ A Practical course that helps me build Machine Learning pipeline in AWS.                                        │
│ NEO4J-powered LLM chatbot with Python Certification- 2024                                                       │
│ A course provided by Neo4j to help me build a LLM-powered backend system using knowledge graphs.                │
│ AWARDS                                                                                                          │
│ Hello GPT-4o Hackathon Certificate of Completion - Top 32/168 2024                                              │
│ Assistants API and Trulens Hackathon Certificate of Completion - Top 26/103 2024                                │
│ Second Prize - Mathematics in English at district level 2016                                                    │
│ Consolation Prize - Hanoi Open Math Competition 2016                                                            │
│                                                                                                                 │
│ You must strictly follow the below format for this task:                                                        │
│ [                                                                                                               │
│   {                                                   

╭───────────────────────────────────────────────────── INFO ──────────────────────────────────────────────────────╮
│ Calling prompt_initialize                                                                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── INFO ──────────────────────────────────────────────────────╮
│ Calling one_shot_prompt                                                                                         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── OUTPUT_MESSAGE ─────────────────────────────────────────────────╮
│ You are tasked to help me generate a dataset of 11 rows entirely in Vietnamese, based entirely on the following │
│ context:                                                                                                        │
│ - PHAM KINH QUOC                                                                                                │
│ +81 80-4721-4151• phamkinhquoc2002@gmail.com • https://www.linkedin.com/in/kinh-quốc-phạm-512642219             │
│ • https://github.com/phamkinhquoc2002                                                                           │
│ INTRODUCTION                                                                                                    │
│ A data science and AI enthusiast with experience in Data Engineering, MLOPs andLLM. Seeking an internship       │
│ or full-time position in the field ofArtificial Intelligenceand Data Engineering, both in applied science and   │
│ research.                                                                                                       │
│ EDUCATION                                                                                                       │
│ Hanoi Highschool for Gifted Students Graduate in May 2020                                                       │
│ Tokyo International University D.B.I Graduate in August 2024                                                    │
│ Relevant Coursework: Big Data, Python in Data Science, Statistics, AI Products and Business Development.        │
│ PROFESSIONAL EXPERIENCE                                                                                         │
│ Tokushukai System Information, Osaka, Japan:AI Engineer November 2024 – Now                                     │
│ • Built theRAG systemintegratingreasoning frameworks,multi-agent collaborationandtool-calling                   │
│ agents for the company, from research, optimization to deployment operations.                                   │
│ • Aligned LLMs throughDPO, GRPO, SFT, enabling efficientlong-context summarizationof medical                    │
│ records, leverageddistributed trainingmethods likeFSDP, DDPfor scalability.                                     │
│ • Optimized and deployed AI systems usingDocker, Google Cloud infrastructure, Kestra, Grafana,                  │
│ Prometheus.                                                                                                     │
│ NAL Solution, Tokyo, Japan:Software Engineer Intern April 2024 – June 2024                                      │
│ • Worked on a team as a back-end developer to build a web app for job hunting, which won the best award for     │
│ "Best Internship Projects", beat 4 other teams.                                                                 │
│ • Used NestJS to create aCRUD pipeline andPostGre-SQL to build the back-end database.                           │
│ ArchivePDF, United States (Online):Data Scientist July 2022 – March 2024                                        │
│ • Performed Sentiment Analysisfor topic classification usingscikit-learn and nltk.                              │
│ • Analyzed and pre-process data, performedanomaly detection.                                                    │
│ Hidden Archive, Online:Data Scientist April 2022 - December 2023                                                │
│ • Applied Correlation Analysis, BERT model andTopic Modelingusing nltk and seaborn, pandasto                    │
│ conduct market research for the company’s exhibition "Archivism 2022".                                          │
│ PROJECTS                                                                                                        │
│ End-to-End Alignment Finetuning: DPO, GRPO and SFT                                                              │
│ A Python-based command-line interface for fine-tuning,

╭───────────────────────────────────────────────────── INFO ──────────────────────────────────────────────────────╮
│ You are tasked to help me generate a dataset of 11 rows entirely in Vietnamese, based entirely on the following │
│ context:                                                                                                        │
│ - SPECIALIZATION                                                                                                │
│ • LLM: Retrieval Augmented Generation, Multi-Agentic Collabora                                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── OUTPUT_MESSAGE ─────────────────────────────────────────────────╮
│                                                                                                                 │
│ ---------DATA_GENERATE---------                                                                                 │
│                                                                                                                 │
│ Sample output of batch 0:```json                                                                                │
│ [                                                                                                               │
│   {                                                                                                             │
│     "prompt": "Hãy đóng vai một nhà quản lý nhân sự phỏng vấn một ứng viên cho vị trí kỹ sư LLM. Hỏi về kinh    │
│ nghiệm của ứng viên trong các lĩnh vực như Retrieval Augmented Generation, Mu                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

TypeError: Subscripted generics cannot be used with class and instance checks